## Getting coordinates

In [ ]:
import requests
import time
import json
import sys
import os

base_url = 'https://nominatim.openstreetmap.org/search'

cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", 
          "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", 
          "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", 
          "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", 
          "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", 
          "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", 
          "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

coordinates = {}

headers = {
    'User-Agent': 'KayakProject/1.0 (emailfortheproject@gmail.com)'
}
# ↖️ A must (so not banned) by Nominatim

for city in cities:
    try:                            # ↙️ Added France so that it don't fetch any homonyms in the world
        params = {'q': f"{city}, France", 'format': 'json', 'limit': 1}
                    # ↖️ Used q for unstructured data as the locations are not of same type (some cities, some specific places near cities etc)
        response = requests.get(base_url, params=params, headers=headers)
        
        response.raise_for_status() 
        
        data = response.json()
        
        if data:  # data is a list with one big item at index 0. So just need to put data[0] to access it
            coordinates[city] = {
                'lat': float(data[0]['lat']),
                'lon': float(data[0]['lon'])
            }
            print(f"{city}: {coordinates[city]}")
        else:
            coordinates[city] = None
            print(f"No results for {city}")
            
    except Exception as e:
        print(f"Error fetching {city}: {e}", file=sys.stderr)
        coordinates[city] = None                # ↖️ don't want my stream to be polluted 
                                                # I prefered it to be in a separate one
    
       # ↙️: Nominatim limits us to 1 request per second
    time.sleep(1)

with open('coordinates.json', 'w') as f:
    json.dump(coordinates, f, indent=4)

print(f"\nCoordinates saved to coordinates.json in {os.getcwd()}")

In [1]:
### Explanation

In [2]:
import requests
import json

base_url = 'https://nominatim.openstreetmap.org/search'
headers = {'User-Agent': 'MyApp/1.0'}

# Example for Lyon
payload = {'q': 'Lyon', 'format': 'json', 'limit': 1}

# STEP 1: requests.get() returns a Response object
result = requests.get(base_url, params=payload, headers=headers) # THIS WILL OUTPUT a very long LIST
print("STEP 1 - result is a Response object:")
print(f"Type: {type(result)}")
print(f"Status code: {result.status_code}")
print(f"Response object: {result}")
print("-" * 50)

# STEP 2: .json() converts the response to Python objects (Decodes and takes what is needed)
data = result.json()
print("STEP 2 - data after .json():")
print(f"Type: {type(data)}")  # <class 'list'>
print(f"Content: {data}")
print("-" * 50)

# What data looks like:
# [
#   {
#     "place_id": 123456,
#     "lat": "45.764043",
#     "lon": "4.835659",
#     "display_name": "Lyon, France",
#     ... more fields ...
#   }
# ]

# STEP 3: Access the first item in the list
if data:  # Check if list is not empty
    first_result = data[0]
    print("STEP 3 - first_result (first item in the list):")
    print(f"Type: {type(first_result)}")  # <class 'dict'>
    print(f"Content: {first_result}")
    print("-" * 50)
    
    # Extract specific fields
    lat = first_result['lat']
    lon = first_result['lon']
    print(f"Extracted - lat: {lat}, lon: {lon}")
    print(f"Types - lat: {type(lat)}, lon: {type(lon)}")  # <class 'str'>

# STEP 4: Store in coordinates dictionary
coordinates = {}
coordinates['Lyon'] = {
    'lat': float(data[0]['lat']),  # Convert string to float
    'lon': float(data[0]['lon'])
}

print("\nSTEP 4 - coordinates dictionary:")
print(f"Type: {type(coordinates)}")  # <class 'dict'>
print(f"Content: {coordinates}")

STEP 1 - result is a Response object:
Type: <class 'requests.models.Response'>
Status code: 200
Response object: <Response [200]>
--------------------------------------------------
STEP 2 - data after .json():
Type: <class 'list'>
Content: [{'place_id': 81817520, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 120965, 'lat': '45.7578137', 'lon': '4.8320114', 'class': 'boundary', 'type': 'administrative', 'place_rank': 16, 'importance': 0.7489604368049191, 'addresstype': 'city', 'name': 'Lyon', 'display_name': 'Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, France', 'boundingbox': ['45.7073666', '45.8082628', '4.7718132', '4.8984245']}]
--------------------------------------------------
STEP 3 - first_result (first item in the list):
Type: <class 'dict'>
Content: {'place_id': 81817520, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relatio

## Getting Weather info (OpenWeather)

In [33]:
# In weather.py
import json

def get_coordinates():
    with open('coordinates.json', 'r', encoding='utf-8') as f:
        return json.load(f)

coordinates = get_coordinates()  # Loading coordinates from coordinates.json
coordinates['Mont Saint Michel']['lat']  # Quick test/example 


48.6359541

In [60]:
import requests
import os
                # It gave us forecast for two days counting the present day
base_url = 'https://api.openweathermap.org/data/2.5/forecast'
# base_url = 'https://api.openweathermap.org/data/2.5/forecast/daily' error 401 - not in range of free tier
api_key = '36f115b32bd1f55fc152ca1d6beacb1d'

all_forecasts = {}
            # WAIT 2 hours for key to be fully activated ?
for place_name, lat_lon_keys in coordinates.items():
    lat = lat_lon_keys['lat']
    lon = lat_lon_keys['lon']

    payload = {'lat': lat, 
              'lon': lon, 
              # 'cnt': 40, not needed to be coded as it is 
              'APPID': api_key,
              'units': 'metric'
              }

    response = requests.get(base_url, params = payload)

    if response.status_code == 200:
        print(f"Forecast for {place_name}:")
        print(response.json())
        all_forecasts[place_name] = response.json()
    else:
        print(f"Error for {place_name}: {response.status_code} - {response.text}")


# Save all forecasts to a JSON file
with open('weather_forecasts.json', 'w') as json_file:
    json.dump(all_forecasts, json_file, indent=2)

print(f"\nCoordinates saved to weather_forecast.json in {os.getcwd()}")


Forecast for Mont Saint Michel:
{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1771243200, 'main': {'temp': 8.12, 'feels_like': 3.76, 'temp_min': 8.12, 'temp_max': 8.62, 'pressure': 1006, 'sea_level': 1006, 'grnd_level': 1002, 'humidity': 80, 'temp_kf': -0.5}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': {'all': 100}, 'wind': {'speed': 9.85, 'deg': 281, 'gust': 15.17}, 'visibility': 10000, 'pop': 0.85, 'rain': {'3h': 0.46}, 'sys': {'pod': 'd'}, 'dt_txt': '2026-02-16 12:00:00'}, {'dt': 1771254000, 'main': {'temp': 8.18, 'feels_like': 3.68, 'temp_min': 8.18, 'temp_max': 8.34, 'pressure': 1007, 'sea_level': 1007, 'grnd_level': 1001, 'humidity': 80, 'temp_kf': -0.16}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': {'all': 100}, 'wind': {'speed': 10.55, 'deg': 287, 'gust': 15.79}, 'visibility': 10000, 'pop': 1, 'rain': {'3h': 0.87}, 'sys': {'pod': 'd'}, 'dt_txt': '2026-02-16 15:00:00

## Getting Booking info